In [976]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from collections import defaultdict
warnings.filterwarnings("ignore")


In [977]:
# Tratamento de dados para o problema

df_dados = pd.read_excel("./sample_data/dados.xlsx")

quantidade_itens = df_dados['Item'].nunique()
quantidade_caixas = df_dados['Caixa Id'].nunique()

itens = df_dados['Item'].unique().tolist()
caixas = df_dados['Caixa Id'].unique().tolist()

pecas_por_caixa = df_dados.groupby('Caixa Id')['Peças'].sum()
pecas_por_item = df_dados.groupby('Item')['Peças'].sum()
pecas_por_item = pecas_por_item.to_dict()

df_dados['Quantidade Total Peças'] = df_dados['Caixa Id'].map(pecas_por_caixa)

df_dados = df_dados.groupby('Caixa Id').agg(list)

def obter_valores_unicos(valores):
  return valores[0] if valores else None

df_dados['Quantidade Total Peças'] = df_dados['Quantidade Total Peças'].apply(obter_valores_unicos)

df_dados = df_dados.sort_values(by='Quantidade Total Peças', ascending=False)

In [978]:
# Variáveis necessárias
np.random.seed(42)
limite_iteracoes_sem_melhora = 10
capacidade_onda = 2000

ondas = dict()

In [979]:
# Solução Inicial: Construtiva gulosa, seleciona as caixas com mais itens distintos e aloca em uma onda.
# Quando a onda estiver sem capacidade para mais itens, ativa nova onda

def gera_solucao_inicial():
  id_onda = 0
  onda = [0] * quantidade_caixas
  ondas[id_onda] = onda

  for caixa in caixas:
    qpecas = df_dados.loc[caixa]['Quantidade Total Peças']
    id_caixa = caixas.index(caixa)

    demanda_onda = obter_capacidade_onda(onda)
    demanda_temporaria = demanda_onda + qpecas
    if demanda_temporaria <= capacidade_onda :
      onda[id_caixa] = 1
    else:
      id_onda += 1
      onda = [0] * quantidade_caixas
      ondas[id_onda] = onda
  return ondas

In [980]:
def calcula_ondas_ativas(solucao):
  ondas_ativas = []
  for id, onda in solucao.items():
    caixas_ativas = [i for i, valor in enumerate(onda) if valor == 1]
    if len(caixas_ativas) < 1:
      ondas_ativas.append(id)
  return ondas_ativas

In [981]:
def define_matriz_solucao(solucao):
  n_ondas = list(solucao.keys())[-1] + 1
  ondas_ativas = calcula_ondas_ativas(solucao)
  n_ondas_ativas = len(ondas_ativas)

  z_kj = np.zeros((quantidade_itens, n_ondas_ativas))

  for id in ondas_ativas:
    for caixa in caixas:
      id_caixa = caixas.index(caixa)
      onda = solucao[id]

      if onda[id_caixa] == 0:
        continue

      itens_caixa = df_dados.loc[caixa]['Item']

      for item in itens_caixa:
        id_item = itens.index(item)
        z_kj[id_item][id] = 1
  return z_kj

In [982]:
# Função Objetivo
def calcula_fo(s):
  return np.sum(s) / quantidade_itens

In [983]:
def obter_capacidade_onda(onda):
  capacidade = 0
  for caixa in caixas:
    id_caixa = caixas.index(caixa)
    if onda[id_caixa] == 0:
      continue

    qpecas = df_dados.loc[caixa]['Quantidade Total Peças']
    capacidade += qpecas
  return capacidade

In [984]:
# Vizinhança
def define_vizinhos(solucao, n_vizinhanca, numero_ondas):
  solucao_vizinha = solucao
  id_onda_selecionada = np.random.randint(0, numero_ondas)
  onda = solucao[id_onda_selecionada]

  caixas_removidas = np.random.choice(caixas, size=n_vizinhanca, replace=False)
  caixas_restantes = n_vizinhanca

  for caixa in caixas_removidas:
    id_caixa = caixas.index(caixa)
    onda[id_caixa] = 0

  for id, onda in solucao_vizinha.items():
    if id == id_onda_selecionada:
      continue

    for caixa in caixas_removidas:
      demanda_onda = obter_capacidade_onda(onda)
      qpecas = df_dados.loc[caixa]['Quantidade Total Peças']
      demanda_temporaria = demanda_onda + qpecas

      if demanda_temporaria <= capacidade_onda:
        id_caixa = caixas.index(caixa)
        onda[id_caixa] = 1
        caixas_restantes -= 1
        solucao_vizinha[id] = onda

  while caixas_restantes > 0:
    numero_ondas += 1
    onda = [0] * quantidade_caixas
    for caixa in caixas_removidas:
      demanda_onda = obter_capacidade_onda(onda)
      qpecas = df_dados.loc[caixa]['Quantidade Total Peças']
      demanda_temporaria = demanda_onda + qpecas

      if demanda_temporaria <= capacidade_onda:
        id_caixa = caixas.index(caixa)
        onda[id_caixa] = 1
        caixas_restantes -= 1

    solucao_vizinha[numero_ondas] = onda
  return solucao_vizinha

In [985]:
def swap(caixas_removidas, onda):
  for caixa in caixas_removidas:
    onda[caixa] = 1
  return onda

def seleciona_onda(id_i, solucao):
  n_ondas = list(solucao.keys())[-1]
  id_j = np.random.randint(0, n_ondas)
  while id_j == id_i:
    id_j = np.random.randint(0, n_ondas)
  return id_j

def seleciona_caixas(onda, quantidade):
  caixas_ativas = [i for i, valor in enumerate(onda) if valor == 1]
  if len(caixas_ativas) >= quantidade:
    return np.random.choice(caixas_ativas, size=quantidade, replace=False)
  return None

def desaloca_caixas(onda, caixas):
  for caixa in caixas:
    onda[caixa] = 0
  return onda

In [986]:
def gera_vizinho(solucao, vizinho):
  if vizinho == 1:
    return obter_2_opt(solucao)
  else:
    return obter_3_opt(solucao)

def obter_2_opt(solucao):
  numero_ondas = list(solucao.keys())[-1]
  numero_tentativas = numero_ondas

  while True:
    id_i = np.random.randint(0, numero_ondas)
    onda_i = solucao[id_i].copy()
    caixas_i = seleciona_caixas(onda_i, 2)

    id_j = seleciona_onda(id_i, solucao)
    onda_j = solucao[id_j].copy()
    caixas_j = seleciona_caixas(onda_j, 2)

    if caixas_i is not None and caixas_j is not None:
      onda_i = desaloca_caixas(onda_i, caixas_i)
      onda_j = desaloca_caixas(onda_j, caixas_j)
      quantidade_caixas_alocadas = len(caixas_i)

      if quantidade_caixas_alocadas > 0:
          break

    numero_tentativas -= 1
    if numero_tentativas <= 0:
      return None

  onda_i = swap(caixas_j, onda_i)
  onda_j = swap(caixas_i, onda_j)

  capacidade_j = obter_capacidade_onda(onda_j)
  capacidade_i = obter_capacidade_onda(onda_i)
  if capacidade_j <= capacidade_onda and capacidade_i <= capacidade_onda:
    solucao[id_i] = onda_i
    solucao[id_j] = onda_j
    quantidade_caixas_alocadas = 0

  while True:
    if quantidade_caixas_alocadas <= 0:
      break

    onda = [0] * quantidade_caixas
    id = numero_ondas + 1
    for caixa in caixas_i:
      capacidade = obter_capacidade_onda(onda)

      if capacidade >= capacidade_onda:
        break

      quantidade_caixas_alocadas -= 1
      onda[caixa] = 1
    solucao[id] = onda

  return solucao

def obter_3_opt(solucao):
  numero_ondas = list(solucao.keys())[-1]
  numero_tentativas = numero_ondas

  while True:
    id_i = np.random.randint(0, numero_ondas)
    onda_i = solucao[id_i].copy()
    caixas_i = seleciona_caixas(onda_i, 3)

    id_j = seleciona_onda(id_i, solucao)
    onda_j = solucao[id_j].copy()
    caixas_j = seleciona_caixas(onda_j, 3)

    if caixas_i is not None and caixas_j is not None:
      onda_i = desaloca_caixas(onda_i, caixas_i)
      onda_j = desaloca_caixas(onda_j, caixas_j)
      quantidade_caixas_alocadas = len(caixas_i)

      if quantidade_caixas_alocadas > 0:
          break

    numero_tentativas -= 1
    if numero_tentativas <= 0:
      return None

  onda_i = swap(caixas_j, onda_i)
  onda_j = swap(caixas_i, onda_j)

  capacidade_j = obter_capacidade_onda(onda_j)
  capacidade_i = obter_capacidade_onda(onda_i)
  if capacidade_j <= capacidade_onda and capacidade_i <= capacidade_onda:
    solucao[id_i] = onda_i
    solucao[id_j] = onda_j
    quantidade_caixas_alocadas = 0

  while True:
    if quantidade_caixas_alocadas <= 0:
      break

    onda = [0] * quantidade_caixas
    id = numero_ondas + 1
    for caixa in caixas_i:
      if quantidade_caixas_alocadas <= 0:
        break

      capacidade = obter_capacidade_onda(onda)

      if capacidade <= capacidade_onda:
        quantidade_caixas_alocadas -= 1
        onda[caixa] = 1

    solucao[id] = onda

  return solucao

In [987]:
# Busca Local
def busca_local(solucao_inicial):
  solucao = solucao_inicial
  z_kj = define_matriz_solucao(solucao)
  iteracoes = 10
  parar_plato = 0

  vizinhos = []

  for vizinho in range(0, quantidade_caixas):
    numero_ondas = list(solucao_inicial.keys())[-1]
    solucao_vizinha = define_vizinhos(solucao, vizinho, numero_ondas)
    vizinhos.append(solucao_vizinha)

  while iteracoes > 0:
    for solucao_vizinha in vizinhos:
      z_kj_vizinho = define_matriz_solucao(solucao_vizinha)

      atual = calcula_fo(z_kj)
      melhor = atual
      solucao_atual = z_kj_vizinho
      fo_vizinho = calcula_fo(solucao_atual)

      if fo_vizinho <= melhor:
        parar_plato = parar_plato + 1 if fo_vizinho == melhor else 0
        melhor = fo_vizinho
        solucao = solucao_vizinha
        z_kj = define_matriz_solucao(solucao_vizinha)

      iteracoes -= 1
    if melhor == atual and np.array_equal(z_kj_vizinho, z_kj) or parar_plato == 20:
      if parar_plato == 20: print('plato')
      break

  return solucao


In [988]:
# VNS
def VNS():
  solucao = gera_solucao_inicial()
  vizinhancas = 2
  iteracoes_sem_melhora = 0

  while iteracoes_sem_melhora < limite_iteracoes_sem_melhora:
    vizinho = 1
    while vizinho <= vizinhancas:
      solucao_vizinha = gera_vizinho(solucao, vizinho)
      solucao_busca_local = busca_local(solucao_vizinha)

      if solucao_busca_local is None:
        iteracoes_sem_melhora += 1

      z_kj_vizinho = define_matriz_solucao(solucao_vizinha)
      z_kj_busca = define_matriz_solucao(solucao_busca_local)

      fo_vizinho = calcula_fo(z_kj_vizinho)
      fo_busca = calcula_fo(z_kj_busca)

      if fo_busca < fo_vizinho:
        solucao = solucao_busca_local
        vizinhos = 1
      else:
        vizinhos += 1
        iteracoes_sem_melhora += 1
  return solucao


In [989]:
def imprime_solucao(df_solucao):
  df_solucao.drop_duplicates(inplace=True)
  grouped_df = df_solucao.groupby('Onda')['Quantidade'].sum().reset_index()
  grouped_df = grouped_df.sort_values(by='Quantidade')

  cores = plt.cm.get_cmap('viridis', len(grouped_df))

  plt.figure(figsize=(10, 6))
  bars = plt.bar(grouped_df['Onda'], grouped_df['Quantidade'], color=cores(np.arange(len(grouped_df))))

  plt.xlabel('Onda')
  plt.ylabel('Quantidades')
  plt.title('Métrica de Ondas')
  plt.xticks(grouped_df['Onda'])

  sm = plt.cm.ScalarMappable(cmap=cores)
  sm.set_array([])
  cbar = plt.colorbar(sm)
  cbar.set_label('Quantidade')

  plt.tight_layout()
  plt.show()

In [990]:
def exportar_solucao(df_solucao):
  excel_file = './sample_data/ondas.xlsx'
  df_solucao.to_excel(excel_file, index=False)

  print(f"Matriz salva com sucesso em '{excel_file}'.")

In [991]:
solucao = VNS()
z_kj = define_matriz_solucao(solucao)
fo_solucao = calcula_fo(z_kj)

print(f"FO encontrada: {fo_solucao}")

df_solucao = pd.DataFrame(columns=['Onda', 'Item', 'Quantidade'])
for id, onda in solucao.items():
  for id_caixa in range(0, quantidade_caixas):
    if onda[id_caixa] == 0:
      continue

    caixa = caixas[id_caixa]

    itens_caixa = df_dados.loc[caixa]['Item']
    pecas_caixa = df_dados.loc[caixa]['Peças']
    for item, peca in zip(itens_caixa, pecas_caixa):
      row = {'Onda': id, 'Item': item, 'Quantidade': peca}
      row = pd.DataFrame([row])
      df_solucao = pd.concat([df_solucao, row], ignore_index=True)

imprime_solucao(df_solucao)
exportar_solucao(df_solucao)

KeyboardInterrupt: 